# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [360]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [368]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [369]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's define the variables:

*  $x_1$: Litres used of the first component.
*  $x_2$: Litres used of the second component.

Our objective function, representing the quality of the product to be maximized, is:

$\max \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$

The constraints are:

*  $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$, stability constraint for the first component.
*  $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$, stability constraint for the second component.
*  $x_1 + x_2 \leq 80$, tank capacity constraint. 
*  $x_1 \geq 0$, non-negativity constraint for the first component.
*  $x_2 \geq 0$, non-negativity constraint for the second component. 

This constitutes the mathematical optimization model for the problem. 


## 3. Generate the pyomo code

In [370]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [371]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define constraints
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)
model.capacity = Constraint(expr=model.x1 + model.x2 <= 80)

# Solve the model
solver = SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)
print("Optimal Solution:")
print("x1 =", value(model.x1))
print("x2 =", value(model.x2))
print("Objective Function Value:", value(model.obj))
```

This code defines the model, variables, objective function, and constraints as specified in your problem. It then uses the 'ipopt' solver to find the optimal solution. 

**Note:** This code provides a basic framework. You might need to adjust the solver or its settings depending on the specific requirements of your problem and the solvers available in your environment. 


## 4. Input problem data and try running the generated code

In [372]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=maximize)

# Define constraints
model.stability1 = Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)
model.capacity = Constraint(expr=model.x1 + model.x2 <= 80)

# Solve the model
solver = SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)
print("Optimal Solution:")
print("x1 =", value(model.x1))
print("x2 =", value(model.x2))
print("Objective Function Value:", value(model.obj))

Status: ok
Termination Condition: optimal
Optimal Solution:
x1 = 43.162277233647174
x2 = 36.8377229631074
Objective Function Value: 2.4827783318454744


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [366]:
print(response.text)

Let's define the variables:

*  $x_1$: Litres used of the first component.
*  $x_2$: Litres used of the second component.

Our objective is to maximize the quality of the product, which is:

 $\text{Maximize} \ln(x_1  x_2) - \sqrt[3]{x_1 + 2x_2}$

We have the following constraints:

* **Constraint 1:** The squared difference between the litres employed of each component and the mean litres utilised of both components is below ten.
 $(x_1 - \frac{x_1+x_2}{2})^2 \le 10$
 $(x_2 - \frac{x_1+x_2}{2})^2 \le 10$

* **Constraint 2:** The mixture tank can only fit 80 l.
 $x_1 + x_2 \le 80$

* **Constraint 3:** Non-negativity of the variables.
 $x_1 \ge 0$
 $x_2 \ge 0$

This leads to the following mathematical optimization model:

$\text{Maximize} \ln(x_1  x_2) - \sqrt[3]{x_1 + 2x_2}$

 $\text{Subject to:}$
 $(x_1 - \frac{x_1+x_2}{2})^2 \le 10$
 $(x_2 - \frac{x_1+x_2}{2})^2 \le 10$
 $x_1 + x_2 \le 80$
 $x_1 \ge 0$
 $x_2 \ge 0$

This model can be solved using optimization algorithms to find the o

In [367]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.x1 = pyo.Var(bounds=(0, None))
model.x2 = pyo.Var(bounds=(0, None))

# Define the objective function
model.objective = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define the constraints
model.constraint_1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.constraint_2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)
model.constraint_3 = pyo.Constraint(expr=model.x1 + model.x2 <= 80)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1 =", pyo.value(model.x1))
print("x2 =", pyo.value(model.x2))
```

This code defines the optimization problem in Pyomo, including the objective function, variables, and constraints. It then uses the 'ipopt' solver 